# 資料合併-短程

In [10]:
import os
import pandas as pd

def merge_files(file_paths, output_path):
    data_frames = []
    
    # 逐一讀取檔案
    for file_path in file_paths:
        if os.path.exists(file_path):
            print(f"正在讀取檔案：{file_path}")
            df = pd.read_csv(file_path)
            if not df.empty:
                data_frames.append(df)
            else:
                print(f"檔案 {file_path} 為空，跳過。")
        else:
            print(f"檔案 {file_path} 不存在，跳過。")

    # 合併所有資料
    if data_frames:
        merged_data = pd.concat(data_frames, ignore_index=True)
        merged_data.to_csv(output_path, index=False, encoding='utf-8-sig')
        print(f"合併完成，結果儲存至：{output_path}")
    else:
        print("沒有任何資料可供合併。")

# 主程式
base_path = '/Users/yuchingchen/Documents/專題/cleaned_data/data/short'
output_path = '/Users/yuchingchen/Documents/專題/cleaned_data/short_flight.csv'

# 檔案名稱清單
file_paths = [
    f'{base_path}/tokyo.csv',
    f'{base_path}/tokyo_business.csv',
    f'{base_path}/hongkong.csv',
    f'{base_path}/hongkong_business.csv',
    f'{base_path}/singapore.csv',
    f'{base_path}/bangkok.csv',
    f'{base_path}/seoul.csv',
    f'{base_path}/seoul_business.csv'
]

# 合併檔案
merge_files(file_paths, output_path)

正在讀取檔案：/Users/yuchingchen/Documents/專題/cleaned_data/data/short/tokyo.csv
正在讀取檔案：/Users/yuchingchen/Documents/專題/cleaned_data/data/short/tokyo_business.csv
正在讀取檔案：/Users/yuchingchen/Documents/專題/cleaned_data/data/short/hongkong.csv
正在讀取檔案：/Users/yuchingchen/Documents/專題/cleaned_data/data/short/hongkong_business.csv
正在讀取檔案：/Users/yuchingchen/Documents/專題/cleaned_data/data/short/singapore.csv
正在讀取檔案：/Users/yuchingchen/Documents/專題/cleaned_data/data/short/bangkok.csv
正在讀取檔案：/Users/yuchingchen/Documents/專題/cleaned_data/data/short/seoul.csv
正在讀取檔案：/Users/yuchingchen/Documents/專題/cleaned_data/data/short/seoul_business.csv
合併完成，結果儲存至：/Users/yuchingchen/Documents/專題/cleaned_data/short_flight.csv


In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

#-------------------------------------------------------------
# 1. 資料
#-------------------------------------------------------------
# 以下為範例資料，請依實際狀況調整
data_short = {
    'Airport': ['日本羽田HND', '仁川ICN', '新加坡SIN', '泰國BKK', '香港HKG', '日本成田NRT', '金浦GMP'],
    'Passengers': [78719302, 71169722, 58946000, 51699104, 39452633, 32705995, 21566946],  # 範例數字
    'Movements': [464910, 337299, 328000, 294085, 276000, 209927, 134560]              # 範例數字
}

df_short = pd.DataFrame(data_short)

#-------------------------------------------------------------
# 2. 封裝一個函式，方便對「長程」或「短程」資料重複執行
#-------------------------------------------------------------
def analyze_airports(df, group_name):
    print(f"\n=== {group_name} 機場資料 ===")
    print(df)

    # 2.1 特徵欄位
    features = ['Passengers', 'Movements']
    X = df[features].values
    
    # 2.2 標準化
    scaler = StandardScaler()
    X_std = scaler.fit_transform(X)
    
    # 2.3 PCA（此處 n_components=2）
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(X_std)
    
    df['PC1'] = X_pca[:, 0]
    df['PC2'] = X_pca[:, 1]
    
    print(f"\n{group_name}：加入 PCA 後的資料")
    print(df)
    
    # 2.4 以 PC1 做簡易分類
    def classify_by_pc1(pc1):
        if pc1 > 1:
            return '大型機場'
        elif pc1 > 0:
            return '中型機場'
        else:
            return '小型機場'
    
    df['Classification_PC1'] = df['PC1'].apply(classify_by_pc1)
    
    # 2.5 K-means 分群 (三群為例)
    kmeans = KMeans(n_clusters=3, random_state=42)
    df['Cluster'] = kmeans.fit_predict(X_pca)
    
    # 命名分群結果（可根據實際觀察再重新命名）
    cluster_map = {
        0: 'Cluster A',
        1: 'Cluster B',
        2: 'Cluster C'
    }
    df['Cluster_Label'] = df['Cluster'].map(cluster_map)
    
    print(f"\n{group_name}：最終資料（含 PC1, PC2, 與兩種分類方式）")
    print(df[['Airport', 'Passengers', 'Movements', 'PC1', 'PC2', 
              'Classification_PC1', 'Cluster_Label']])
    
    # 回傳結果 DataFrame（若後續要做更多分析，可再利用）
    return df

#-------------------------------------------------------------
# 3. 對短程做分析
#-------------------------------------------------------------
df_short_result = analyze_airports(df_short, "短程")


=== 短程 機場資料 ===
   Airport  Passengers  Movements
0  日本羽田HND    78719302     464910
1    仁川ICN    71169722     337299
2   新加坡SIN    58946000     328000
3    泰國BKK    51699104     294085
4    香港HKG    39452633     276000
5  日本成田NRT    32705995     209927
6    金浦GMP    21566946     134560

短程：加入 PCA 後的資料
   Airport  Passengers  Movements       PC1       PC2
0  日本羽田HND    78719302     464910  2.304679  0.230734
1    仁川ICN    71169722     337299  1.089987 -0.426968
2   新加坡SIN    58946000     328000  0.570849 -0.044270
3    泰國BKK    51699104     294085  0.054708 -0.025752
4    香港HKG    39452633     276000 -0.529727  0.293328
5  日本成田NRT    32705995     209927 -1.263336  0.057469
6    金浦GMP    21566946     134560 -2.227159 -0.084542

短程：最終資料（含 PC1, PC2, 與兩種分類方式）
   Airport  Passengers  Movements       PC1       PC2 Classification_PC1  \
0  日本羽田HND    78719302     464910  2.304679  0.230734               大型機場   
1    仁川ICN    71169722     337299  1.089987 -0.426968               大型機場   
2   新

# 短程航班

In [1]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

# --------------------------
# 1. 載入資料及初步處理
# --------------------------
data_path = '/Users/yuchingchen/Documents/專題/cleaned_data/short_flight.csv'
data = pd.read_csv(data_path)

# 統一「出發日期」格式為 "YYYY-MM-DD"
data["出發日期"] = pd.to_datetime(
    data["出發日期"], 
    format="%Y-%m-%d", 
    errors="coerce"
).combine_first(
    pd.to_datetime(data["出發日期"], format="%Y/%m/%d", errors="coerce")
)
data["出發日期"] = data["出發日期"].dt.strftime("%Y-%m-%d")

# --------------------------
# 2. 計算對數變數與新增最低價格天數
# --------------------------
data["平均價格_log"] = data["平均價格"].apply(lambda x: np.log1p(x) if pd.notnull(x) else np.nan).round(2)
data["最低價格_log"] = data["最低價格"].apply(lambda x: np.log1p(x) if pd.notnull(x) else np.nan).round(2)
data["價格變異_log"] = data["價格變異"].apply(lambda x: np.log1p(x) if pd.notnull(x) else np.nan).round(2)
data["中位數價格_log"] = data["中位數價格"].apply(lambda x: np.log1p(x) if pd.notnull(x) else np.nan).round(2)

# 新增最低價格天數 (91 - '最低價格剩餘天數')
data['最低價格天數'] = 91 - data['最低價格剩餘天數']

# 移除包含 NaN 的數據（針對關鍵欄位）
data = data.dropna(subset=["平均價格_log", "最低價格_log", "最低價格天數", "價格變異_log", "中位數價格_log"])

# 顯示抵達機場代號種類及數量
print(data['抵達機場代號'].value_counts())

# --------------------------
# 3. 假期判斷
# --------------------------
holidays = {
    "台灣": [("2025-01-20", "2025-02-02"), ("2025-02-28", "2025-03-02")],
    "日本": [("2025-02-11", "2025-02-11"), ("2025-02-22", "2025-02-24"), ("2025-03-20", "2025-03-20")],
    "香港": [("2025-01-29", "2025-01-31")],
    "新加坡": [("2025-01-29", "2025-01-30")],
    "韓國": [("2025-01-28", "2025-01-30"), ("2025-03-01", "2025-03-03")]
}

airport_to_region = {
    "HKG": "香港",
    "NRT": "日本",
    "HND": "日本",
    "BKK": "泰國",  # 泰國沒有假期，不處理
    "ICN": "韓國",
    "GMP": "韓國",
    "SIN": "新加坡"
}

# 將「出發日期」轉回 datetime 格式以利比較
data["出發日期"] = pd.to_datetime(data["出發日期"])

def is_holiday(row):
    # 台灣假期判斷（以出發地為台灣）
    for start, end in holidays["台灣"]:
        if pd.to_datetime(start) <= row["出發日期"] <= pd.to_datetime(end):
            return 1
    # 根據抵達機場對應地區判斷假期
    region = airport_to_region.get(row["抵達機場代號"])
    if region and region in holidays:
        for start, end in holidays[region]:
            if pd.to_datetime(start) <= row["出發日期"] <= pd.to_datetime(end):
                return 1
    return 0

data["假期"] = data.apply(is_holiday, axis=1)

# 將「假期」欄位移至「艙等」欄位之後（若「艙等」存在）
cols = data.columns.tolist()
if '假期' in cols and '艙等' in cols:
    cols.insert(cols.index('艙等') + 1, cols.pop(cols.index('假期')))
data = data[cols]

# --------------------------
# 4. 新增額外的地區經濟資料
# --------------------------
additional_data = {
    "Region": ["香港", "日本", "新加坡", "韓國", "泰國"],
    "Cost of Living Index": [70.8, 46.1, 76.7, 60.1, 34.1],
    "GDP (PPP) per capita (in thousand USD)": [71.627, 51.399, 141.553, 60.046, 23.981],
}
region_df = pd.DataFrame(additional_data)

data["Region"] = data["抵達機場代號"].map(airport_to_region)
data = data.merge(region_df, how="left", on="Region")

# --------------------------
# 5. 新增是否為平日欄位
# --------------------------
weekend_days = ["週六", "週日"]
data["是否為平日"] = data["星期"].apply(lambda x: 1 if x in weekend_days else 0)

# 將「是否為平日」欄位移至「是否過夜」欄位之後
cols = data.columns.tolist()
if '是否過夜' in cols and '是否為平日' in cols:
    idx = cols.index("是否過夜") + 1
    cols.insert(idx, cols.pop(cols.index("是否為平日")))
data = data[cols]

# --------------------------
# 6. 新增機場分類欄位
# --------------------------
super_large_airports = ['HND', 'ICN']
largr_airports = ['SIN', 'BKK']
medium_large_airports = ['HKG', 'GMP', 'NRT']

data['機場分類'] = data['抵達機場代號'].apply(
    lambda x: 1 if x in super_large_airports else (2 if x in medium_large_airports else None)
).astype('Int64')

# 將「機場客運量分類」移至「抵達機場代號」後
cols = data.columns.tolist()
if '抵達機場代號' in cols and '機場分類' in cols:
    arrival_idx = cols.index('抵達機場代號')
    cols.insert(arrival_idx + 1, cols.pop(cols.index('機場分類')))
data = data[cols]

# --------------------------
# 7. 新增「飛行時間_分鐘」欄位
# --------------------------
def convert_flight_time_to_minutes(time_str):
    """
    將 'x 小時 y 分鐘' 格式的字串轉換為分鐘數
    """
    if isinstance(time_str, str):
        match = re.match(r'(\d+)\s*小時(?:\s*(\d+)\s*分鐘)?', time_str)
        if match:
            hours = int(match.group(1))
            minutes = int(match.group(2)) if match.group(2) else 0
            return hours * 60 + minutes
    return None

data['飛行時間_分鐘'] = data['飛行時間'].apply(convert_flight_time_to_minutes)

# 將「飛行時間_分鐘」移至「飛行時間」欄位之後
cols = data.columns.tolist()
if '飛行時間' in cols and '飛行時間_分鐘' in cols:
    flight_idx = cols.index('飛行時間')
    cols.insert(flight_idx + 1, cols.pop(cols.index('飛行時間_分鐘')))
data = data[cols]

# --------------------------
# 8. 新增競爭航班數欄位
# --------------------------
# 此處依據「出發機場代號」、「抵達機場代號」、「出發日期」與「出發時段」來計算每一組別中航班的數量
data['competing_flights'] = data.groupby(
    ['出發機場代號', '抵達機場代號', '出發日期', '出發時段']
)['航空公司'].transform('size')

data = data.sort_values(
    by=['出發機場代號', '抵達機場代號', '出發日期', '出發時段', '航空公司']
)

# --------------------------
# 9. 匯出處理後的資料
# --------------------------
output_path = '/Users/yuchingchen/Documents/專題/cleaned_data/short_flight_final.csv'
data.to_csv(output_path, index=False)

抵達機場代號
HKG    3485
NRT    1937
BKK    1578
ICN    1226
SIN    1051
HND    1013
GMP     169
Name: count, dtype: int64
